In [1]:
%pylab notebook
import cv2
import time

Populating the interactive namespace from numpy and matplotlib


# Load the Haar Cascades

In [2]:
face_cascade  = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
eye_cascade   = cv2.CascadeClassifier("haarcascade_eye_tree_eyeglasses.xml")

In [3]:
# Function   : set_effect
# Input      : img    - video frame
#              option - set of all effects to be performed
# Output     : output frame
# Description: Perform the requested effect on the video frame
def set_effect(img, option):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 3)
    for (x,y, w, h) in faces:
        roi_g = gray[y:y+h, x:x+w]
        roi_c = img[y:y+h, x:x+w]
        

        eye = eye_cascade.detectMultiScale(roi_g, 1.1, 3)

    return img            

# Start the video capture


In [4]:
test = cv2.VideoCapture(0)
option = set([])

In [5]:
while(True):
    
    ret, frame = test.read()

    try:
        out = set_effect(frame, option)
        font = cv2.FONT_HERSHEY_SCRIPT_COMPLEX
        cv2.putText(out,'Change Effect',(400,465), font, 1.2,(127,127,255),2,cv2.LINE_AA)
        out = cv2.resize(out, (1200, 900))
        cv2.resizeWindow('frame', 1200,900)
        cv2.imshow('frame',out)
    except:
        pass
    
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
test.release()
cv2.destroyAllWindows()

In [6]:
test.release()
cv2.destroyAllWindows()